# Import Docs

In [1]:
from pathlib import Path
import shutil

# Import your existing function
from scripts.decode_refs import replace_bible_refs

In [2]:
RAW_DIR = Path("docs/raw_notes")
CLEAN_DIR = Path("docs/cleaned_notes")

# Create output directory if it doesn't exist
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

RAW_DIR, CLEAN_DIR

(PosixPath('docs/raw_notes'), PosixPath('docs/cleaned_notes'))

In [3]:
raw_files = [p for p in RAW_DIR.rglob("*") if p.is_file()]

print(f"Found {len(raw_files)} files:")
for p in raw_files[:10]:
    print(" -", p)

if len(raw_files) > 10:
    print("...")

Found 10 files:
 - docs/raw_notes/ECAL1___T2___L14.md
 - docs/raw_notes/ECAL1___T2___L2.md
 - docs/raw_notes/GE___T2___L9.md
 - docs/raw_notes/BOC1___T2___L7.md
 - docs/raw_notes/GOW___T2___L11.md
 - docs/raw_notes/GOW___T2___L7.md
 - docs/raw_notes/FM___T3___L11.md
 - docs/raw_notes/ECAL2___T4___L13.md
 - docs/raw_notes/GE___T4___L2.md
 - docs/raw_notes/GOW___T2___L17.md


In [4]:
def process_file(src_path: Path, dst_root: Path):
    """
    Apply replace_bible_refs to a single file
    and write it to the mirrored location in dst_root.
    """
    # Preserve subdirectory structure
    relative_path = src_path.relative_to(RAW_DIR)
    dst_path = dst_root / relative_path

    # Ensure parent directory exists
    dst_path.parent.mkdir(parents=True, exist_ok=True)

    # Read → transform → write
    text = src_path.read_text(encoding="utf-8")
    transformed = replace_bible_refs(text)
    dst_path.write_text(transformed, encoding="utf-8")

    return dst_path


In [5]:
processed = []

for src in raw_files:
    dst = process_file(src, CLEAN_DIR)
    processed.append(dst)

print(f"Processed {len(processed)} files.")


Processed 10 files.


In [6]:
# Pick one example file
example = raw_files[0]
cleaned = CLEAN_DIR / example.relative_to(RAW_DIR)

print("=== RAW ===")
print(example.read_text(encoding="utf-8")[:500])

print("\n=== CLEANED ===")
print(cleaned.read_text(encoding="utf-8")[:500])


=== RAW ===
Hymns:: #593, #407 
Speaker:: #mniu
Subject:: Dealing with the Spirit (spirit)

-
- Message
	- Verse Recitation
		- ((b048c006-a001-b1b1-b1b1-b1b1b1b1b1b1)): all the brothers that need this memorize. I hope you take this to heart.
		- ((b060c003-a004-b1b1-b1b1-b1b1b1b1b1b1)): all the sisters that need this memorize. This verse comes in a context for sisters, the adornment. Brother Lee somewhere has a really marvelous message (or fn.) on the matter of shame-fastness(?) in the eyes of God. This is

=== CLEANED ===
Hymns:: #593, #407 
Speaker:: #mniu
Subject:: Dealing with the Spirit (spirit)

-
- Message
	- Verse Recitation
		- <BIBLE_REF book="Galatians" chapter="6" verse_start="1" verse_end="1">Galatians 6:1</BIBLE_REF>: all the brothers that need this memorize. I hope you take this to heart.
		- <BIBLE_REF book="1 Peter" chapter="3" verse_start="4" verse_end="4">1 Peter 3:4</BIBLE_REF>: all the sisters that need this memorize. This verse comes in a context for sisters, the

In [7]:
import re

BIBLE_REF_TAG = re.compile(r"<BIBLE_REF\b")

count = 0
for p in processed:
    count += len(BIBLE_REF_TAG.findall(p.read_text(encoding="utf-8")))

print(f"Total Bible references tagged: {count}")


Total Bible references tagged: 91


# Convert links to verse references

In [1]:
import scripts.decode_refs as dr

source = "The author of Hebrews quotes the Psalms in ((b058c002-a012-b1b1b1b1))."

print(dr.replace_bible_refs(source))

The author of Hebrews quotes the Psalms in <BIBLE_REF book="Hebrews" chapter="2" verse_start="12" verse_end="12">Hebrews 2:12</BIBLE_REF>.


In [2]:
source = "Some verse ranges ((b047c007-a001-b1b1-b1b1-b1b1b1b1b1b1))-3."

print(dr.replace_bible_refs(source))

Some verse ranges <BIBLE_REF book="2 Corinthians" chapter="7" verse_start="1" verse_end="3">2 Corinthians 7:1–3</BIBLE_REF>.
